# Spend single P2PKH input - BX Exercise
In this exercise, we spend a single P2PKH input to a P2PKH output.

<hr style="border: 0.5px dashed #000;">

## 1. Restore wallet and generate spending + receiving key pairs.


### Restore HD master keys from mnemonic sentence.

In [4]:
# my_mnemonic ="word0 word1 word2 ..."
# hd_master_private=$( )
# hd_master_public=$( )

my_mnemonic="amount right cheese defy click eight slight strategy replace earn simple labor"
hd_master_private=$(bx mnemonic-to-seed $my_mnemonic | bx hd-new)
hd_master_public=$(bx hd-to-public $hd_master_private)

### Restore your first receiving key in your account 1: `m/44'/1'/1'/0/0`.

(We will be spending the funds controlled by this key)

In [5]:
hd_m_44h_1h_1h_0_0=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 1  \
| bx hd-private --index 0 \
| bx hd-private --index 0)

privatekey_44h_1h_1h_0_0=$(bx hd-to-ec $hd_m_44h_1h_1h_0_0)
publickey_44h_1h_1h_0_0=$(bx hd-to-public $hd_m_44h_1h_1h_0_0 | bx hd-to-ec)
publickeyhash_44h_1h_1h_0_0=$(bx sha256 $publickey_44h_1h_1h_0_0 | bx ripemd160)
address_44h_1h_1h_0_0=$(bx hd-to-public $hd_m_44h_1h_1h_0_0 | bx hd-to-ec | bx ec-to-address --version 111)

echo $address_44h_1h_1h_0_0
echo $publickeyhash_44h_1h_1h_0_0


mo8WTrRtUfQofjwekDE19LuKJrZG1tGKou
53824add0669e6b702a1171b8bef72a43ac013ce


### Fetch previous UTXO you are spending.


In [6]:
# bx fetch-tx [transaction_hash] -f json | jq ".transaction.outputs[output_index]"

bx fetch-tx 86ad635645920497c233cc33556463fa6258ab8f9c5f7d12748a6638216e3363 -f json | jq ".transaction.outputs[0]"

{
  "address_hash": "53824add0669e6b702a1171b8bef72a43ac013ce",
  "script": "dup hash160 [53824add0669e6b702a1171b8bef72a43ac013ce] equalverify checksig",
  "value": "499543"
}


In [7]:
# previous_txid=
# previous_output_index=
# previous_output_amount=

previous_txid=86ad635645920497c233cc33556463fa6258ab8f9c5f7d12748a6638216e3363
previous_output_index=0
previous_output_amount=499543


### Account 2: Generate destination key pair.

You will spend the previously received utxo to receiving addresses from `account 2`, namely `/44'/1'/2'/0/0`.

In [8]:
# Derive hd private key m_44h_1h_2h_0_0 from your master keys:

# hd_m_44h_1h_2h_0_0=
# publickeyhash_44h_1h_2h_0_0=

hd_m_44h_1h_2h_0_0=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 2  \
| bx hd-private --index 0 \
| bx hd-private --index 0)

publickeyhash_44h_1h_2h_0_0=$(bx hd-to-public $hd_m_44h_1h_2h_0_0 | bx hd-to-ec | bx sha256 | bx ripemd160)
echo $publickeyhash_44h_1h_2h_0_0

736b4005d4999db71c7184f7c716c43d6f896854


## 2. Spending a P2PKH output.

In [9]:
# TODO: Show image of spending process with BX.
# Encode transaction with outputs, input points, version, locktime.


### Create output scripts.

* **Compose P2PKH(destination public key hash) output scripts for each destination.**

In [10]:
# bx script-encode "DUP HASH160 [public key hash] EQUALVERIFY CHECKSIG"

# output_script=

output_script=$(bx script-encode "DUP HASH160 ["$publickeyhash_44h_1h_2h_0_0"] EQUALVERIFY CHECKSIG")
echo $output_script

76a914736b4005d4999db71c7184f7c716c43d6f89685488ac


### Compute output amount(s).

* **Subtract minimum fee, which consists of:**
    * Example minimum fee levels: `100 sat / sigop + 1 sat / byte`
    * This is node policy. 


* **Transaction sigop count computation:**
    * Counted as 1 sigop: `checksig`,`checksigverify`
    * Counted as 20 sigops: `checkmultisig`, `checkmultisigverify`
    * *Non-segwit:*
        * Transaction sigop count is multiplied by 4x.
        * Output vs Input sigops: 
            * All sigops in the output script are counted.
            * P2SH Redeemscript input sigops are counted.
      

In [11]:
# Sigops: 1 outputs, so total of 1 sigops.
# tx bytes: 4 + 1 + 1*(32+4+110+4) + 1 + 1*(8+33) + 4 
tx_byte_count=$(expr 4 + 1 + 1 \* 150 + 1 + 1 \* 41 + 4)
echo $tx_byte_count


201


In [12]:
# Output amount total.
output_amount=$(expr $previous_output_amount - 1 \* 400 - $tx_byte_count \* 1)
echo $output_amount


498942


### Construct transaction template for signing.

* `sequence: 0xffffffff(hex)/4294967295(dec)`

In [14]:
# bx tx-encode \
# --input [previous tx id]:[index]:[sequence] \
# --output [output script]:[output amount]

# my_tx=

my_tx=$(bx tx-encode \
--input $previous_txid:$previous_output_index:4294967295 \
--output $output_script:$output_amount)


### Sign transaction.

**Fetch previous output script**

In [16]:
# bx fetch-tx --format json [previous tx id] 

# previous_output_script=

previous_output_script=$(bx fetch-tx --format json $previous_txid \
| jq -r ".transaction.outputs[0].script")
echo $previous_output_script


dup hash160 [53824add0669e6b702a1171b8bef72a43ac013ce] equalverify checksig


**Sign transaction with previous output script.**

In [17]:
# bx input-sign [private key] "previous output script" [transaction template]

# signature= 

signature=$(bx input-sign --sign_type all --index 0 $privatekey_44h_1h_1h_0_0 "$previous_output_script" $my_tx)


**Set the input script into the finalised transaction.**

In [18]:
# bx input-set "[signature] [public key point]" [transaction template]

# my_tx=

my_tx=$(bx input-set --index 0 "[$signature] [$publickey_44h_1h_1h_0_0]" $my_tx)


## 3. Validate & Broadcast the endorsed transaction

In [20]:
bx validate-tx $my_tx

The transaction is valid.


In [21]:
bx send-tx $my_tx

Sent transaction.


In [22]:
# bx bitcoin256 [tx data]
bx bitcoin256 $my_tx


94f4fdbb22667eeaffac2f7767e5b413ec1aff221733e513e97de03551a2c633
